<a href="https://colab.research.google.com/github/CarbonErmiyasdaniel/plant-disease-detection-vgg16/blob/main/plant_disease_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install tensorflow gradio -q

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
import tensorflow_datasets as tfds

# Load the beans dataset
(train_ds, val_ds, test_ds), info = tfds.load(
    "beans",
    split=['train', 'validation', 'test'],
    as_supervised=True,
    with_info=True
)

print("Classes:", info.features['label'].names)
print("Training samples:", info.splits['train'].num_examples)


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def format_image(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0  # normalize
    return image, label

train_ds = train_ds.map(format_image).batch(BATCH_SIZE).shuffle(1000)
val_ds = val_ds.map(format_image).batch(BATCH_SIZE)
test_ds = test_ds.map(format_image).batch(BATCH_SIZE)


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # 3 classes
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Train Acc')
plt.plot(val_acc, label='Val Acc')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Train Loss')
plt.plot(val_loss, label='Val Loss')
plt.legend()
plt.title('Loss')

plt.show()


In [ ]:
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print(classification_report(y_true, y_pred, target_names=info.features['label'].names))


In [ ]:
model.save("plant_disease_vgg16.h5")
print("✅ Model saved as plant_disease_vgg16.h5")


In [ ]:
import tensorflow as tf
import numpy as np
import gradio as gr

# Reload model if needed
model = tf.keras.models.load_model("plant_disease_vgg16.h5")  # Or skip if already in memory

# Define constants
IMG_SIZE = (224, 224)
class_names = ['angular_leaf_spot', 'bean_rust', 'healthy']

# Descriptions for better output
descriptions = {
    "angular_leaf_spot": "Angular Leaf Spot: This is a fungal disease. The leaves have brown spots with yellow edges. It often happens in humid conditions. Recommended action: Improve air circulation and consider fungicide treatment.",
    "bean_rust": "Bean Rust: This is a fungal infection that shows orange or rust-colored spots on the leaves. It reduces the plant's ability to make food. Recommended action: Remove infected leaves and apply fungicides.",
    "healthy": "Healthy Leaf: The leaf looks good with no signs of disease. Keep up the good care!"
}

def predict_bean(image):
    try:
        # Ensure image is valid
        if image is None:
            return "No image uploaded.", "", ""

        img = tf.image.resize(image, IMG_SIZE)
        img = tf.expand_dims(img, 0) / 255.0
        pred = model.predict(img)
        result = class_names[np.argmax(pred)]
        confidence = float(np.max(pred))
        description = descriptions[result]
        return result, f"{confidence*100:.2f}%", description
    except Exception as e:
        return "Error", "Error", str(e)

demo = gr.Interface(
    fn=predict_bean,
    inputs=gr.Image(type="numpy", label="Upload a bean leaf image"),
    outputs=[
        gr.Textbox(label="Prediction"),
        gr.Textbox(label="Confidence"),
        gr.Textbox(label="Description")
    ],
    title="🌿 Bean Disease Detection (VGG16 Transfer Learning)",
    description="Upload a bean leaf image to detect if it's healthy or diseased. The system identifies Angular Leaf Spot, Bean Rust, or Healthy leaves."
)

demo.launch()





In [ ]:
print("✅ Project Summary")
print("----------------------------")
print("Model: VGG16 Transfer Learning")
print("Dataset: TensorFlow Beans (3 classes)")
print("Performance Metric: Accuracy, F1-score")
print("Demo: Gradio Web Interface")
print("Training completed successfully!")
